In [1]:
from sqlalchemy import create_engine

engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

## Connection

우리가 데이터베이스에 붙어서 작업하기 위해서는 **Connection** 객체가 필요하다. 공식문서에서도 나와있듯이 Engine 의 목적은 데이터베이스와 연결되기 위해 커넥션 객체를 제공하는 것이다.
Connection 객체는 데이터베이스에 접근하기 위해 필요한 자원을 제공해주고, 우리가 특정 범위안에서 자원을 정리할 수 있도록 도와주기도 한다. 이러한 범위를 제공해주기 위해서는 파이썬 언어에서 지원하는
[`contextmanager`](https://docs.python.org/3/reference/datamodel.html#context-managers) 기능을 [`with`](https://docs.python.org/3/reference/compound_stmts.html#with) 와 함께 사용하면 편하다.

아래 예시를 보면 `conn` 객체(contextmanager) 가 트랜잭션의 자원을 열고 해당 범위안에서 작업이 끝나자. `ROLLBACK` 시켜버린것을 확인할 수 있습니다.
여기서 ROLLBACK 당하지 않기 위해서는 꼭 commit 을 해주어야 합니다. (혹은 auto_commit 옵션을 켜주셔야 합니다)

In [2]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(text("select 'Hello World'"))
    print(result.all())

2025-04-25 00:03:30,561 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-25 00:03:30,562 INFO sqlalchemy.engine.Engine select 'Hello World'
2025-04-25 00:03:30,563 INFO sqlalchemy.engine.Engine [generated in 0.00221s] ()
[('Hello World',)]
2025-04-25 00:03:30,564 INFO sqlalchemy.engine.Engine ROLLBACK


In [3]:
with engine.connect() as conn:
    """데이터를 생성하고 커밋하는 예제"""
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(text("INSERT INTO some_table (x, y) VALUES (:x, :y)"), [{"x": 1, "y": 2}, {"x": 3, "y": 4}])
    conn.commit()

2025-04-25 00:03:30,571 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-25 00:03:30,572 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2025-04-25 00:03:30,573 INFO sqlalchemy.engine.Engine [generated in 0.00173s] ()
2025-04-25 00:03:30,574 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2025-04-25 00:03:30,574 INFO sqlalchemy.engine.Engine [generated in 0.00048s] [(1, 2), (3, 4)]
2025-04-25 00:03:30,575 INFO sqlalchemy.engine.Engine COMMIT


## Commit as you go

우리가 위의 예시에서 블럭안에 있는 코드들에 대한 작업을 완료했음을 나타내기 위해 **커밋(Commit)**을 수행한 것을 확인할 수 있습니다.
SQLAlchemy 에서는 위와 같이 여러 복잡한 SQL Statements 를 수행하면서 작업이 완료될때마다 **커밋(Commit)** 을 해나가는 것을 **"Commit as you go"** 라는 패턴이라고 부릅니다.
즉, 조금은 개발자가 언제 **커밋(Commit)**을 할지 개입해야 하는 부분이 존재하게 됩니다.

## Begin once

백엔드 작업에서는 **Commit as you go** 패턴으로 인식되는 경우가 대부분이므로 SQLAlchemy 에서는 편의 기능인 `begin()` 메소드를 제공합니다. 이 메소드를 사용하면, 커넥션을 자동으로 얻어오는 것 이외에 
자동으로 해당 범위의 작업이 성공적으로 끝나면 `COMMIT` 까지 수행해줍니다. 만약, 예외가 발생한다면 `ROLLBACK` 도 진행해줍니다. 다만, 아래 예시의 실행결과의 첫줄 보면 알수 있듯이 `BEGIN (implicit)` 라는 뜻은 
데이터 베이스에는 **어떠한 명령(command)을 보내지 않았고**, 논리적으로 트랜잭션이 시작됬다는 표시만 진행합니다. 실제 트랜잭션은 이후 커맨드가 진행될때 부터 시작됩니다. 따라서 개발자 개인의 역량에 기대기 보다는 조금 더 자동화된 트랜잭션 시스템을 구축할 수 있습니다.

In [4]:
with engine.begin() as conn:
    conn.execute(text("INSERT INTO some_table (x, y) VALUES (:x, :y)"), [{"x": 6, "y": 7}, {"x": 8, "y": 9}])
    raise Exception("test")

2025-04-25 00:03:30,582 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-25 00:03:30,583 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2025-04-25 00:03:30,583 INFO sqlalchemy.engine.Engine [cached since 0.009627s ago] [(6, 7), (8, 9)]
2025-04-25 00:03:30,584 INFO sqlalchemy.engine.Engine ROLLBACK


Exception: test

In [5]:
with engine.begin() as conn:
    conn.execute(text("INSERT INTO some_table (x, y) VALUES (:x, :y)"), [{"x": 10, "y": 20}, {"x": 30, "y": 40}])
    conn.commit()

2025-04-25 00:00:13,047 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-25 00:00:13,049 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2025-04-25 00:00:13,050 INFO sqlalchemy.engine.Engine [cached since 9.264s ago] [(10, 20), (30, 40)]
2025-04-25 00:00:13,051 INFO sqlalchemy.engine.Engine COMMIT


In [8]:
"""
롤백된 데이터가 안들어간것 확인 (6,7), (8,9) 데이터가 롤백된것 확인
"""
with engine.begin() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    for row in result:
        print(f"x: {row.x}, y: {row.y}")

2025-04-25 00:02:22,486 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-25 00:02:22,488 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2025-04-25 00:02:22,489 INFO sqlalchemy.engine.Engine [cached since 126.4s ago] ()
x: 1, y: 2
x: 3, y: 4
x: 1, y: 2
x: 3, y: 4
x: 10, y: 20
x: 30, y: 40
2025-04-25 00:02:22,490 INFO sqlalchemy.engine.Engine COMMIT
